In [0]:
#imported libraries 
import pandas as pd
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC, LinearSVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import Perceptron
from sklearn.linear_model import SGDClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import AdaBoostClassifier
import itertools 

from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn import metrics
from sklearn.pipeline import Pipeline

In [0]:
#combined training and testing sets to have entire dataset to ensure features are the
#same in both train and test sets
# Code to read csv file into colaboratory:
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# 1. Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

#2. Get the file
downloaded = drive.CreateFile({'id':'16N6CwZ16PBfQf1Jj2zEwSBq3bKpYi0gP'}) # replace the id with id of file you want to access
downloaded.GetContentFile('train.csv')
downloaded = drive.CreateFile({'id':'1deEEIhUO5NJGebfWNfS0rzBtZhoPB8zh'})
downloaded.GetContentFile('test.csv')


#3. Read file as panda dataframe
train_df = pd.read_csv('train.csv')
test_df = pd.read_csv('test.csv')
combine = [train_df, test_df]    

In [3]:
type(combine)

list

In [4]:
# 1400 is the median so we choose that as the cutoff point of whether an article
# is popular or not, for the classification model we are about to build
# (we could play with this value to make "popularity" more/less exclusive)
for dataset in combine:
    dataset.loc[ dataset[' shares'] <= 1400, ' shares'] = 0
    dataset.loc[ dataset[' shares'] > 1400, ' shares'] = 1
    
# look at first five rows to check that "shares" is now a boolean column
# 0 = unpopular, 1 = popular
train_df.head()

,url,timedelta,n_tokens_title,n_tokens_content,n_unique_tokens,n_non_stop_words,n_non_stop_unique_tokens,num_hrefs,num_self_hrefs,num_imgs,...,min_positive_polarity,max_positive_polarity,avg_negative_polarity,min_negative_polarity,max_negative_polarity,title_subjectivity,title_sentiment_polarity,abs_title_subjectivity,abs_title_sentiment_polarity,shares
0,http://mashable.com/2013/06/06/social-makeover...,581.0,9.0,645.0,0.523659,1.0,0.705722,9.0,4.0,1.0,...,0.033333,0.8,-0.249653,-0.4,-0.125,0.066667,0.033333,0.433333,0.033333,0
1,http://mashable.com/2013/03/18/gears-of-war-ju...,661.0,11.0,2104.0,0.376261,1.0,0.557325,25.0,3.0,11.0,...,0.100000,1.0,-0.275221,-0.8,-0.050,0.100000,0.000000,0.400000,0.000000,0
2,http://mashable.com/2014/09/17/ios-8-arrives-h...,113.0,10.0,540.0,0.580786,1.0,0.649852,30.0,2.0,1.0,...,0.033333,0.8,-0.300000,-0.8,-0.050,0.900000,0.500000,0.400000,0.500000,0
3,http://mashable.com/2014/09/22/twitter-app-how...,107.0,10.0,711.0,0.530670,1.0,0.668113,21.0,7.0,2.0,...,0.033333,0.8,-0.251282,-0.5,-0.050,0.100000,0.200000,0.400000,0.200000,0
4,http://mashable.com/2014/07/31/doctor-who-prem...,161.0,13.0,796.0,0.473418,1.0,0.624242,12.0,0.0,1.0,...,0.100000,0.6,-0.194753,-0.6,-0.050,0.400000,0.100000,0.100000,0.100000,1


In [0]:
# X sets have only input variables, Y sets have only target variable of shares
X_train = train_df.drop(['url', ' shares'],axis=1) #X_train is the training data withour the 1st and 61st column
Y_train = train_df[' shares']                      #Y_train only contains our target variable
X_test = test_df.drop(['url', ' shares'],axis=1)   #X_test = "Original_data - X_train"
Y_test = test_df[' shares']                        #Y_test is used to verify the accuracy against what we predict later.

In [0]:
#############################
# added
###############

pca=PCA(n_components=2)
newtrain_df = pca.fit_transform(train_df.drop(['url', ' shares'],axis=1))
newtest_df = pca.fit_transform(test_df.drop(['url', ' shares'],axis=1))

In [0]:
####################
# added
#########
features_train=pd.DataFrame(newtrain_df)
features_test=pd.DataFrame(newtest_df)

In [0]:
###########
# added
##########
cls=LogisticRegression()


pipe = Pipeline([('pca', pca), ('logistic', cls)])
pipe.fit(features_train, train_df[' shares'])
predictions = pipe.predict(features_test)

In [9]:
print(metrics.confusion_matrix(Y_test, predictions))
print(metrics.classification_report(Y_test, predictions)) 
acc_log = round(metrics.accuracy_score(Y_test, predictions) * 100, 2) #Calculating the subset accuracy.
prec_log = round(metrics.precision_score(Y_test, predictions) * 100, 2) #Calculating the precision, which is the ability of a model not to label as positive a sample that is negative. 
recall_log = round(metrics.recall_score(Y_test, predictions) * 100, 2) #Calculating the recall, which is the ability of the model to find all the positive samples.
f1score_log = round(metrics.f1_score(Y_test, predictions) * 100, 2)    #Calculating the F1 score which is the weighted average of precision and recall.
auc_log = round(metrics.roc_auc_score(Y_test, predictions) * 100, 2)   #Compute Area Under the Receiver Operating Characteristic Curve (ROC AUC) from prediction scores.


[[2700 1355]
 [2286 1588]]
             precision    recall  f1-score   support

          0       0.54      0.67      0.60      4055
          1       0.54      0.41      0.47      3874

avg / total       0.54      0.54      0.53      7929



In [0]:
cls=AdaBoostClassifier()


pipe = Pipeline([('pca', pca), ('ada', cls)])
pipe.fit(features_train, train_df[' shares'])
Y_pred = pipe.predict(features_test)

In [11]:
# AdaBoost

adaboost = AdaBoostClassifier(n_estimators=100)
adaboost.fit(X_train, Y_train)
Y_pred = adaboost.predict(X_test)
print(metrics.confusion_matrix(Y_test, Y_pred))
print(metrics.classification_report(Y_test, Y_pred))
acc_adaboost = round(metrics.accuracy_score(Y_test, Y_pred) * 100, 2)
prec_adaboost = round(metrics.precision_score(Y_test, Y_pred) * 100, 2)
recall_adaboost = round(metrics.recall_score(Y_test, Y_pred) * 100, 2)
f1score_adaboost = round(metrics.f1_score(Y_test, Y_pred) * 100, 2)
auc_adaboost = round(metrics.roc_auc_score(Y_test, Y_pred) * 100, 2)
train_acc_adaboost = round(adaboost.score(X_train, Y_train) * 100, 2)

[[2765 1290]
 [1360 2514]]
             precision    recall  f1-score   support

          0       0.67      0.68      0.68      4055
          1       0.66      0.65      0.65      3874

avg / total       0.67      0.67      0.67      7929

